# Higher-order functions and a "higher-level perspective" on computation #

_(September 20, 2022)_

- Functions (and lambda functions)
- Properties of slices
- Problem solving example: Longest consecutive subsequences (from Codewars)

## Functions (and lambda functions) ##

Suppose you want to sort a list of numbers. Python's built-in `sorted` function can accomplish this task:

In [1]:
unsorted_list = [1912, 1956, 1906]
### INSERT: SORT ###
sorted(unsorted_list)

[1906, 1912, 1956]

Consider the following "list of dictionaries."

In [2]:
data = [{'first': 'Guido', 'last': 'Van Rossum', 'year': 1956},
        {'first': 'Grace', 'last': 'Hopper',     'year': 1906},
        {'first': 'Alan',  'last': 'Turing',     'year': 1912}]
print(data)

[{'first': 'Guido', 'last': 'Van Rossum', 'year': 1956}, {'first': 'Grace', 'last': 'Hopper', 'year': 1906}, {'first': 'Alan', 'last': 'Turing', 'year': 1912}]


> _Aside:_ You can use the [pprint module](https://docs.python.org/3/library/pprint.html) to "pretty print" basic Python data structures.

In [3]:
from pprint import pprint
pprint(data)

[{'first': 'Guido', 'last': 'Van Rossum', 'year': 1956},
 {'first': 'Grace', 'last': 'Hopper', 'year': 1906},
 {'first': 'Alan', 'last': 'Turing', 'year': 1912}]


Suppose you want to sort this list by year. The following doesn't work; why not?

In [4]:
pprint(data)
#sorted(data)  # Fails: why?

[{'first': 'Guido', 'last': 'Van Rossum', 'year': 1956},
 {'first': 'Grace', 'last': 'Hopper', 'year': 1906},
 {'first': 'Alan', 'last': 'Turing', 'year': 1912}]


Sorting `data` fails because there is no way to compare the values, which are dictionaries. However, `sorted` allows you to specify a _key_ function that returns a value to use for ordering.

```python
# for any `e` in `data`, `key(e)`
# returns a "sortable" value:
sorted(data, key=...)
```

In [5]:
### INSERT: SORT BY KEY ###
def get_year(e): # Let `e` be one of the dictionary elements of the `data` list
    return e['year']

sorted(data, key=get_year)

[{'first': 'Grace', 'last': 'Hopper', 'year': 1906},
 {'first': 'Alan', 'last': 'Turing', 'year': 1912},
 {'first': 'Guido', 'last': 'Van Rossum', 'year': 1956}]

Sorted is an example of a **higher-order function**, that is, a function that accepts another function as input or produces a new function as output. It's helpful because you can implement a sorting procedure once, and let a user customize it later on.

**Lambda (or "anonymous") functions:** Useful for writing functions that you don't expect to reuse. Lambdas create a function value but do not assign it a name.

For example, suppose you want to sort `data` by first name. Rather than create a separate function for extracting the first name, you can implement this function "inline" using the `lambda` construct to create the function you need.

In [6]:
### INSERT: USE LAMBDA TO SORT BY FIRST NAME ###
sorted(data, key=lambda e: e['first'])

[{'first': 'Alan', 'last': 'Turing', 'year': 1912},
 {'first': 'Grace', 'last': 'Hopper', 'year': 1906},
 {'first': 'Guido', 'last': 'Van Rossum', 'year': 1956}]

> Lambdas are convenient but not necessarily easy for a future reader to understand. It's best left to situations where you the function you need is simple or you are sure you won't need to reuse that function in other situations.

## Facts about slicing ##

Take a moment to review how slicing works.

Here is one example: suppose you have the list `L` below and you want to extract the last three elements. Come up with two slice-based methods to do it.

In [7]:
L = [1, 2, 3, 4, 5]

### INSERT: FRONT- AND BACK-RELATIVE SLICES ###
print("Method 0:", L[2:])
print("Method 1:", L[-3:])

Method 0: [3, 4, 5]
Method 1: [3, 4, 5]


When you reference a list using an invalid index, you get an error. Uncomment the following line to see an example:

In [8]:
#L[len(L)] # Fails: Why?

Slices work a little differently. Consider the following slice of a list:

In [9]:
### INSERT: SLICE FROM `len(L)` ###
L[len(L):] ## ??

[]

Instead of it failing, it returns an empty list!

You can think of it this way. First, Python interprets the slice; a range with invalid indices yields an empty range. When you then reference the list on an empty range, you get an empty list.

Here is another example:

In [10]:
L[len(L):1_000_000_000]

[]

## Exercise: Longest consecutive subsequence ##

Given a string of characters, write some code to find longest consecutive substring of repeated characters. It should then return the character and the number of times it occurred in that longest substring. If there are multiple substrings of repeated characters having the same length, then the function should return the first one that occurs.

In [11]:
# Examples:
tests = {
    'aaaabb': ('a', 4),
    'bbbaaabaaaa': ('a', 4),
    'bbbaaaabaaa': ('a', 4),
    'bbbaabbaaa': ('b', 3), # tie: return first
    'cbdeuuu900': ('u', 3),
    'abbbbb': ('b', 5),
    'aabb': ('a', 2),
    'ba': ('b', 1),
    '': ('', 0)
}

Below, we'll implement a few variants. To check them, we'll use the following function.

> It's a _higher-order function!_ It takes an implementation of LCS as input and checks it against the test cases shown above.

In [12]:
def check(implementation):
    for input_string, true_answer in tests.items():
        print(f'* Checking {repr(input_string)}: True solution is {repr(true_answer)}...')
        your_answer = implementation(input_string)
        assert your_answer == true_answer, f'Your code produced {repr(your_answer)} instead.'
    print("\n(Passed!)")

**Exercise.**

In [13]:
def longest_consecutive_subsequence(s):
    pass # return a "letter, count" pair

### Baseline method: One-at-a-time ###

Here is a version you might naturally implement. It works as follows.

1. Let `best` and `best_count` hold the letter with the largest count seen so far.
2. Visit each character `c` of the input string, `s` from left to right.
3. Determine whether `c` is the same as the previous letter, `previous`, which has occurred `previous_count`.
4. If `c` is the same as `previous`, increment `previous_count`.
5. Otherwise, see if we can update `best` with `previous`.

In [14]:
def longest_consecutive_subsequence_0a(s):
    best, best_count = '', 0
    previous, previous_count = '', 0
    for c in s:
        if c == previous:
            previous_count += 1
        else:
            if previous_count > best_count:
                best, best_count = previous, previous_count
            previous, previous_count = c, 1
    if previous_count > best_count:
        best, best_count = previous, previous_count
    return best, best_count

First, let's check this solution:

In [15]:
check(longest_consecutive_subsequence_0a)

* Checking 'aaaabb': True solution is ('a', 4)...
* Checking 'bbbaaabaaaa': True solution is ('a', 4)...
* Checking 'bbbaaaabaaa': True solution is ('a', 4)...
* Checking 'bbbaabbaaa': True solution is ('b', 3)...
* Checking 'cbdeuuu900': True solution is ('u', 3)...
* Checking 'abbbbb': True solution is ('b', 5)...
* Checking 'aabb': True solution is ('a', 2)...
* Checking 'ba': True solution is ('b', 1)...
* Checking '': True solution is ('', 0)...

(Passed!)


The solution shown above has some repetition: the code to update `best` and `best_count` when a new and longer subsequence is discovered appears in two places (lines 8-9 and again in lines 11-12). We'll address that momentarily.

Here is a variation on the above with a few revisions for clarity:

1. _Refactor_ the redundant code into a separate function, `update`. Doing so improves reading, debugging, and maintaining the function. (If you wanted to solve the _shortest_ consecutive subsequence problem instead, you need only change `update`!)
2. Maintain substrings and use `len` to get the length when needed.
3. Use slices to handle empty strings more robustly.

In [16]:
def longest_consecutive_subsequence_0b(s):
    def update(best, previous):
        return max(best, previous, key=len)
    previous = ''
    best = ''
    for c in s:
        if c != previous[-1:]:
            best = update(best, previous)
            previous = ''
        previous += c
    best = update(best, previous)
    return best[-1:], len(best)

check(longest_consecutive_subsequence_0b)

* Checking 'aaaabb': True solution is ('a', 4)...
* Checking 'bbbaaabaaaa': True solution is ('a', 4)...
* Checking 'bbbaaaabaaa': True solution is ('a', 4)...
* Checking 'bbbaabbaaa': True solution is ('b', 3)...
* Checking 'cbdeuuu900': True solution is ('u', 3)...
* Checking 'abbbbb': True solution is ('b', 5)...
* Checking 'aabb': True solution is ('a', 2)...
* Checking 'ba': True solution is ('b', 1)...
* Checking '': True solution is ('', 0)...

(Passed!)


### Method 1: Data parallelism ###

Instead of thinking about building the solution one character at a time, let's try to think about the input "as a whole." What are you really looking for?

In the case of this problem, the input string is really a collection of substrings having the same letter. You can think of finding the solution in two parts: (1) finding the substrings, and then (2) finding the largest one.

For example, consider the input string again:

In [17]:
s = 'bbbaaabaaaa'

The pieces correspond to these slices of `s`:

In [18]:
s[0:3], s[3:6], s[6:7], s[7:11]

('bbb', 'aaa', 'b', 'aaaa')

To reconstruct these substrings, all you need are the locations of _changes_, which occur at positions `[0, 3, 6, 7, 11]`.

To find these, we need to look at consecutive pairs of letters in `s` and see where they differ. Let's write some code to build up this information in two parts.

In [19]:
def zip_neighbors(s):
    ### INSERT CODE ###
    return zip(s[:-1], s[1:])

print("s =", s)
list(zip_neighbors(s))

s = bbbaaabaaaa


[('b', 'b'),
 ('b', 'b'),
 ('b', 'a'),
 ('a', 'a'),
 ('a', 'a'),
 ('a', 'b'),
 ('b', 'a'),
 ('a', 'a'),
 ('a', 'a'),
 ('a', 'a')]

To associate these neighbor-pairs with positions, apply `enumerate`:

In [20]:
print("s =", s)

### INSERT: APPLY ENUMERATE ###
list(enumerate(zip_neighbors(s)))

s = bbbaaabaaaa


[(0, ('b', 'b')),
 (1, ('b', 'b')),
 (2, ('b', 'a')),
 (3, ('a', 'a')),
 (4, ('a', 'a')),
 (5, ('a', 'b')),
 (6, ('b', 'a')),
 (7, ('a', 'a')),
 (8, ('a', 'a')),
 (9, ('a', 'a'))]

These are the building blocks for a "change detector:"

In [21]:
def detect_changes(s):
    ### INSERT SOLUTION ###
    interior_changes = [p+1 for p, (x, y) in enumerate(zip_neighbors(s)) if x != y]
    return [0] + interior_changes + [len(s)]

detect_changes(s)

[0, 3, 6, 7, 11]

> _Aside:_ The implementation of `detect_changes` produces the following result on an empty input:

In [22]:
detect_changes('')

[0, 0]

From the changes, the pieces follow:

In [23]:
def get_pieces(s, changes):
    ### INSERT SOLUTION ###
    return [s[i:j] for i, j in zip_neighbors(changes)]

get_pieces(s, detect_changes(s))

['bbb', 'aaa', 'b', 'aaaa']

The largest piece is then easy to find:

In [24]:
### FIND LARGEST PIECE VIA KEYED `max` ###
max(get_pieces(s, detect_changes(s)), key=len)

'aaaa'

Putting it all together:

In [25]:
def longest_consecutive_subsequence_1(s):
    changes = detect_changes(s)
    pieces = get_pieces(s, changes)
    largest_piece = max(pieces, key=len)
    return largest_piece[:1], len(largest_piece)

longest_consecutive_subsequence_1(s)

('a', 4)

In [26]:
check(longest_consecutive_subsequence_1)

* Checking 'aaaabb': True solution is ('a', 4)...
* Checking 'bbbaaabaaaa': True solution is ('a', 4)...
* Checking 'bbbaaaabaaa': True solution is ('a', 4)...
* Checking 'bbbaabbaaa': True solution is ('b', 3)...
* Checking 'cbdeuuu900': True solution is ('u', 3)...
* Checking 'abbbbb': True solution is ('b', 5)...
* Checking 'aabb': True solution is ('a', 2)...
* Checking 'ba': True solution is ('b', 1)...
* Checking '': True solution is ('', 0)...

(Passed!)


### Method 2: Data parallel "preconditioning" approach ###

Transform the input into something, e.g., preprocess or "precondition" it so that the problem becomes easier to solve.

In [27]:
### INSERT: DEMO `s` and `t` ###
s = 'bbbaaabaaaa'
t = 'bbb|aaa|b|aaaa'
t.split('|')

['bbb', 'aaa', 'b', 'aaaa']

In [28]:
def insert_cuts(s, separator='|'):
    assert separator not in s     # Q for you: "but y tho"
    def conditional_inject(x, y):
        return x + (separator if x != y else '')
    cut_pieces = [conditional_inject(x, y) for x, y in zip_neighbors(s)]
    cut_pieces.append(s[-1:])
    return ''.join(cut_pieces)

def longest_consecutive_subsequence_3(s):
    t = insert_cuts(s)
    substrings = t.split('|')
    largest_substring = max(substrings, key=len)
    return largest_substring[-1:], len(largest_substring)    

check(longest_consecutive_subsequence_3)

* Checking 'aaaabb': True solution is ('a', 4)...
* Checking 'bbbaaabaaaa': True solution is ('a', 4)...
* Checking 'bbbaaaabaaa': True solution is ('a', 4)...
* Checking 'bbbaabbaaa': True solution is ('b', 3)...
* Checking 'cbdeuuu900': True solution is ('u', 3)...
* Checking 'abbbbb': True solution is ('b', 5)...
* Checking 'aabb': True solution is ('a', 2)...
* Checking 'ba': True solution is ('b', 1)...
* Checking '': True solution is ('', 0)...

(Passed!)


## Summary ##

Here is a quick review of the main ideas in this note:

- **Functions** encapsulate ... functionality!
- **Higher-order functions** enable customization, e.g., `sorted` and `max`.
- **Slices** are well-worth mastering. Empty ranges produced empty lists, which we exploited to write more robust code for corner cases.
- **Data parallelism** is an "all-at-once" mindset, which can be more efficient, easier to read, and easier to parallelize.